In [ ]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    # Get the number of available GPUs
    num_gpus = torch.cuda.device_count()
    print(f"Number of available GPUs: {num_gpus}")

    # Optional: print the name of each GPU
    for i in range(num_gpus):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA is not available. No GPUs found.")

Number of available GPUs: 2
GPU 0: Tesla T4
GPU 1: Tesla T4


In [ ]:
! pip install bitsandbytes
! pip install datasets==3.6.0



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 34.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 10.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.7/146.7 kB 11.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.10.0
    Uninstalling fsspec-2025.10.0:
      Successfully uninstalled fsspec-2025.10.0
  Attempting uninstall: dill
    Found existing installation: dill 0.4.0
    Uninstalling dill-0.4.0:
      Successfully uninstalled dill-0.4.0
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.18
    Uninstalling multiprocess-0.70.18:
      Successfully uninstalled multiprocess-0.70.18
  Attempting uninstall: datasets
    Found existing installation: datasets 4.4.2
    Uninsta

In [ ]:
!git config --global credential.helper store
from huggingface_hub import login
login()

In [ ]:
%%writefile train.py
import torch
import pytorch_lightning as pl
from pytorch_lightning.callbacks import Callback
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    PeftModel
)
from datasets import load_dataset
from torch.utils.data import DataLoader
import os
import gc
from torch.optim.lr_scheduler import CosineAnnealingLR, LinearLR, SequentialLR


MODEL_ID = "Codemaster67/OLMo-7B-ZINC20-10k"
NEW_ADAPTER_NAME = "Codemaster67/OLMo-7B-ZINC20-10k"



class OLMoQLoRA(pl.LightningModule):
    def __init__(self, model_id, adapter_name):
        super().__init__()
        self.save_hyperparameters()
        self.model_id = model_id
        self.adapter_name = adapter_name

        self.tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

    def configure_model(self):

        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=True,
        )

        model = AutoModelForCausalLM.from_pretrained(
            self.model_id,
            quantization_config=bnb_config,
            trust_remote_code=True
        )

        model.config.use_cache = False
        model.gradient_checkpointing_enable()
        model = prepare_model_for_kbit_training(model)

        peft_config = LoraConfig(
            r=64,
            lora_alpha=128,
            target_modules="all-linear",
            lora_dropout=0.05,
            bias="none",
            task_type="CAUSAL_LM"
        )
        self.model = get_peft_model(model, peft_config)

        if self.trainer.is_global_zero:
            self.model.print_trainable_parameters()

    def forward(self, input_ids, attention_mask, labels=None):
        return self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

    def training_step(self, batch, batch_idx):
        outputs = self(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=batch["labels"]
        )
        loss = outputs.loss
        self.log("train_loss", loss, prog_bar=True, on_step=True, sync_dist=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=1e-4, weight_decay=1e-4)

        total_steps = self.trainer.estimated_stepping_batches

        warmup_steps = int(0.15 * total_steps)
        scheduler_warmup = LinearLR(
            optimizer,
            start_factor=0.001,
            end_factor=1.0,
            total_iters=warmup_steps,
        )

        scheduler_cosine = CosineAnnealingLR(
            optimizer,
            T_max=total_steps - warmup_steps,
        )

        scheduler = SequentialLR(
            optimizer,
            schedulers=[scheduler_warmup, scheduler_cosine],
            milestones=[warmup_steps]
        )

        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "interval": "step"
            }
        }


class ZINCDataModule(pl.LightningDataModule):
    def __init__(self, tokenizer, batch_size=4):
        super().__init__()
        self.tokenizer = tokenizer
        self.batch_size = batch_size

    def setup(self, stage=None):
        self.dataset = (
            load_dataset("zpn/zinc20", split="train", streaming=True, trust_remote_code=True)
            .take(10_000)
        )

    def train_dataloader(self):
        def collate_fn(batch):
            texts = [item['smiles'] for item in batch]

            encodings = self.tokenizer(
                texts,
                truncation=True,
                max_length=256,
                padding="max_length",
                return_tensors="pt"
            )

            labels = encodings["input_ids"].clone()
            labels[labels == self.tokenizer.pad_token_id] = -100

            return {
                "input_ids": encodings["input_ids"],
                "attention_mask": encodings["attention_mask"],
                "labels": labels
            }

        return DataLoader(
            self.dataset,
            batch_size=self.batch_size,
            collate_fn=collate_fn,
        )

if __name__ == "__main__":
    pl_model = OLMoQLoRA(MODEL_ID, NEW_ADAPTER_NAME)

    dm = ZINCDataModule(pl_model.tokenizer, batch_size=2)

    trainer = pl.Trainer(
        accelerator="gpu",
        devices=2,
        strategy="ddp",
        precision="16-mixed",
        max_epochs=1,
        accumulate_grad_batches=8,
        log_every_n_steps=16,
        enable_checkpointing=False,
        enable_progress_bar=False,
        gradient_clip_val=0.5
    )

    print("Starting Training on first 10k samples...")
    trainer.fit(pl_model, datamodule=dm)

    # ---------------------------------------------------------
    # MERGE AND PUSH LOGIC
    # ---------------------------------------------------------
    if trainer.is_global_zero:
        print("Training complete. Saving adapter to temporary local storage...")

        pl_model.model.save_pretrained("./temp_adapter")
        pl_model.tokenizer.save_pretrained("./temp_adapter")

        # Free up memory (VRAM) to allow loading the full base model for merging
        print("Freeing VRAM for merge process...")
        del pl_model
        del trainer
        del dm
        gc.collect()
        torch.cuda.empty_cache()

        print("Loading Base Model in FP16 for merging...")
        # Load base model in float16 (required for merging, cannot merge into 4bit)
        base_model = AutoModelForCausalLM.from_pretrained(
            MODEL_ID,
            return_dict=True,
            torch_dtype=torch.float16,
            device_map="auto",
            trust_remote_code=True
        )

        print("Loading Adapter and Merging...")
        model_to_merge = PeftModel.from_pretrained(base_model, "./temp_adapter")
        model_to_merge = model_to_merge.merge_and_unload()

        print(f"Pushing FULL MERGED model to Hub: {NEW_ADAPTER_NAME}")
        model_to_merge.push_to_hub(NEW_ADAPTER_NAME)

        # Push tokenizer as well
        tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
        tokenizer.push_to_hub(NEW_ADAPTER_NAME)

        print("Done! Full merged model pushed.")

Overwriting train.py


In [ ]:
!python train.py

2026-01-26 12:37:57.483176: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769431077.723848     448 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769431077.794746     448 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769431078.381900     448 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769431078.381941     448 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769431078.381945     448 computation_placer.cc:177] computation placer alr